# Analyse Runtime and Its Reduction Due To Approximation

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from pathlib import Path

# Set plotly theme
pio.templates.default = "plotly_white"

# Load the data
data_path = Path("/Users/yeva/imperial/master-proj/gradual-sem-causal-aba/results/gradual/v2_ablation_random_graphs_7nodes/cpdag_metrics.csv")
df = pd.read_csv(data_path)

print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")
df.head()

Data shape: (850, 30)
Columns: ['nnz', 'fdr', 'tpr', 'fpr', 'precision', 'recall', 'F1', 'shd', 'sid_low', 'sid_high', 'dataset', 'seed', 'n_nodes', 'n_edges', 'neighbourhood_n_nodes', 'max_cycle_length', 'max_ct_depth', 'max_path_length', 'max_c_set_size', 'search_depth', 'elapsed_bsaf_creation', 'elapsed_model_solution', 'is_converged', 'fact_ranking_method', 'model_ranking_method', 'num_edges_est', 'best_model', 'aba_elapsed', 'ranking_elapsed', 'best_I']

First few rows:


,nnz,fdr,tpr,fpr,precision,recall,F1,shd,sid_low,sid_high,...,elapsed_bsaf_creation,elapsed_model_solution,is_converged,fact_ranking_method,model_ranking_method,num_edges_est,best_model,aba_elapsed,ranking_elapsed,best_I
0,0,0.0,0.0,0.0,NaN,0.0,NaN,7.0,22.0,22.0,...,136.762595,429.702305,True,v2,original_ranking,0,[],0.013444,0.046985,-2.454362
1,0,0.0,0.0,0.0,NaN,0.0,NaN,7.0,22.0,22.0,...,136.762595,429.702305,True,v2,original_ranking,0,[],0.013444,0.093017,-2.454362
2,0,0.0,0.0,0.0,NaN,0.0,NaN,7.0,22.0,22.0,...,136.762595,429.702305,True,v2,original_ranking,0,[],0.013444,0.186441,-2.454362
3,0,0.0,0.0,0.0,NaN,0.0,NaN,7.0,22.0,22.0,...,136.762595,429.702305,True,v2,original_ranking,0,[],0.013444,0.375774,-2.454362
4,0,0.0,0.0,0.0,NaN,0.0,NaN,7.0,22.0,22.0,...,136.762595,429.702305,True,v2,original_ranking,0,[],0.013444,0.757659,-2.454362


In [2]:
df.columns

Index(['nnz', 'fdr', 'tpr', 'fpr', 'precision', 'recall', 'F1', 'shd',
       'sid_low', 'sid_high', 'dataset', 'seed', 'n_nodes', 'n_edges',
       'neighbourhood_n_nodes', 'max_cycle_length', 'max_ct_depth',
       'max_path_length', 'max_c_set_size', 'search_depth',
       'elapsed_bsaf_creation', 'elapsed_model_solution', 'is_converged',
       'fact_ranking_method', 'model_ranking_method', 'num_edges_est',
       'best_model', 'aba_elapsed', 'ranking_elapsed', 'best_I'],
      dtype='object')

In [3]:
[c for c in df.columns if 'elapsed' in c]

['elapsed_bsaf_creation',
 'elapsed_model_solution',
 'aba_elapsed',
 'ranking_elapsed']

In [4]:
# Prepare the data similar to demo.ipynb
df['neighbourhood_n_nodes'] = df['max_cycle_length']
df['use_collider_arguments'] = df['max_ct_depth'] > -1  # Assuming -1 indicates no collider arguments used

# Runtime columns to analyze
runtime_cols = ['elapsed_bsaf_creation', 'elapsed_model_solution', 'aba_elapsed', 'ranking_elapsed']

print("Runtime columns available:")
for col in runtime_cols:
    if col in df.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col} - NOT FOUND")

print(f"\nData shape: {df.shape}")
print(f"Unique parameter values:")
print(f"neighbourhood_n_nodes: {sorted(df['neighbourhood_n_nodes'].unique())}")
print(f"use_collider_arguments: {sorted(df['use_collider_arguments'].unique())}")
print(f"max_c_set_size: {sorted(df['max_c_set_size'].unique())}")
print(f"search_depth: {sorted(df['search_depth'].unique())}")

Runtime columns available:
✓ elapsed_bsaf_creation
✓ elapsed_model_solution
✓ aba_elapsed
✓ ranking_elapsed

Data shape: (850, 31)
Unique parameter values:
neighbourhood_n_nodes: [np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7)]
use_collider_arguments: [np.False_, np.True_]
max_c_set_size: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
search_depth: [np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]


In [5]:
# Function to create runtime ablation plots for a specific parameter
def create_runtime_ablation_plot(param_name, fixed_values_dict=None):
    """
    Create runtime ablation plot for a specific parameter while keeping others fixed.
    
    Args:
        param_name: The parameter to vary ('neighbourhood_n_nodes', 'use_collider_arguments', 'max_c_set_size', 'search_depth')
        fixed_values_dict: Dictionary of fixed values for other parameters
    """
    
    # Default fixed values (same as in demo.ipynb)
    default_fixed = {
        'neighbourhood_n_nodes': 7,
        'use_collider_arguments': True, 
        'max_c_set_size': 5,
        'search_depth': 10
    }
    
    if fixed_values_dict:
        default_fixed.update(fixed_values_dict)
    
    # Remove the varying parameter from fixed values
    varying_param = param_name
    fixed_params = {k: v for k, v in default_fixed.items() if k != varying_param}
    
    # Filter data for fixed parameter values
    filtered_data = df.copy()
    for param, value in fixed_params.items():
        filtered_data = filtered_data[filtered_data[param] == value]
    
    if len(filtered_data) == 0:
        print(f"No data found for the specified fixed parameters: {fixed_params}")
        return None
    
    # Runtime columns to analyze
    runtime_cols = ['elapsed_bsaf_creation', 'elapsed_model_solution', 'aba_elapsed', 'ranking_elapsed']
    
    # Group by the varying parameter and compute statistics
    param_stats = filtered_data.groupby(varying_param)[runtime_cols].agg(['mean', 'std', 'count']).reset_index()
    param_stats.columns = [col[0] if col[1] == '' else f'{col[0]}_{col[1]}' for col in param_stats.columns]
    
    # Create figure with all runtime components
    fig = go.Figure()
    
    # Color scheme for different runtime components
    colors = {
        'elapsed_bsaf_creation': '#1f77b4',
        'elapsed_model_solution': '#ff7f0e', 
        'aba_elapsed': '#2ca02c',
        'ranking_elapsed': '#d62728'
    }
    
    # Add traces for each runtime component
    for runtime_col in runtime_cols:
        if f'{runtime_col}_mean' in param_stats.columns:
            fig.add_trace(
                go.Scatter(
                    x=param_stats[varying_param],
                    y=param_stats[f'{runtime_col}_mean'],
                    error_y=dict(type='data', array=param_stats[f'{runtime_col}_std'], visible=True),
                    mode='lines+markers',
                    name=runtime_col.replace('_', ' ').replace('elapsed ', '').title(),
                    line=dict(width=3, color=colors.get(runtime_col, '#000000')),
                    marker=dict(size=8)
                )
            )
    
    # Update layout
    title_text = f'Runtime Analysis: {varying_param}'
    if fixed_params:
        fixed_str = ', '.join([f'{k}={v}' for k, v in fixed_params.items()])
        title_text += f'<br><sub>Fixed parameters: {fixed_str}</sub>'
    
    fig.update_layout(
        title=title_text,
        xaxis_title=varying_param.replace('_', ' ').title(),
        yaxis_title='Runtime (seconds)',
        height=500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    fig.show()
    
    return param_stats

In [6]:
# Generate all runtime ablation plots

# 1. Neighbourhood N Nodes runtime ablation
print("1. Neighbourhood N Nodes Runtime Ablation")
runtime_stats_cycle = create_runtime_ablation_plot('neighbourhood_n_nodes')

# 2. Use collider arguments runtime ablation
print("\n2. Use Collider Arguments Runtime Ablation")
runtime_stats_ct_depth = create_runtime_ablation_plot('use_collider_arguments')

# 3. Max C Set Size runtime ablation  
print("\n3. Max C Set Size Runtime Ablation")
runtime_stats_c_set = create_runtime_ablation_plot('max_c_set_size')

# 4. Search Depth runtime ablation
print("\n4. Search Depth Runtime Ablation") 
runtime_stats_search = create_runtime_ablation_plot('search_depth')

1. Neighbourhood N Nodes Runtime Ablation



2. Use Collider Arguments Runtime Ablation



3. Max C Set Size Runtime Ablation



4. Search Depth Runtime Ablation


In [9]:
# Runtime statistics for default parameters only
print("Runtime Statistics for Default Parameters")
print("=" * 50)

# Default parameter values (same as in demo.ipynb)
default_params = {
    'neighbourhood_n_nodes': 7,
    'use_collider_arguments': True, 
    'max_c_set_size': 5,
    'search_depth': 10
}

# Filter data for default parameters
default_data = df.copy()
for param, value in default_params.items():
    default_data = default_data[default_data[param] == value]

print(f"Number of runs with default parameters: {len(default_data)}")

# Runtime columns to analyze
runtime_cols = ['elapsed_bsaf_creation', 'elapsed_model_solution', 'aba_elapsed', 'ranking_elapsed']

# Calculate mean and std for each runtime component
runtime_stats = default_data[runtime_cols].agg(['mean', 'std']).round(4)

# Display as a clean table
print(f"\nRuntime Statistics (seconds):")
print(runtime_stats.T.to_string())

Runtime Statistics for Default Parameters
Number of runs with default parameters: 50

Runtime Statistics (seconds):
                            mean      std
elapsed_bsaf_creation   136.7626   0.0000
elapsed_model_solution  466.2075  50.4869
aba_elapsed               0.0155   0.0047
ranking_elapsed           3.8505   1.2080
